In [ ]:
#final scripts used in portfolio project data
#1. the portfolio+index+macroeconomic data
#2. calculate the yield in the index
#3. charts using matplotlob, seaborn

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import missingno as ms
import seaborn as sns
import sqlalchemy
import json
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from datetime import datetime, timedelta
import pandas_datareader.data as pdr
import yfinance as yf

In [2]:
from fredapi import Fred
import openpyxl

# Define o caminho do arquivo
file_path = r'C:\Users\Danielf\Desktop\projects data Daniel\Portfolio manager with python\overview portfolio.xlsx'

# ==== Primeira Página: Dados do Portfólio ====
# Dados do portfólio (mesmo código que você já usou)
stocks = [
    'NVDA', 'AVGO', 'TSM', 'MSFT', 'AMZN', 'GOOGL', 'META', 
    'JPM', 'BAC', 'GS', 'V', 'MA', 'SBUX', 'MCD', 'KO', 
    'WMT', 'BA', 'CVX'
]

purchase_details = {
    'Stock': stocks,
    'Date of Purchase': [
        '2023-01-03', '2023-02-10', '2023-03-07', '2023-01-20', '2023-02-15', 
        '2023-03-10', '2023-01-25', '2023-02-28', '2023-03-15', '2023-01-26', 
        '2023-02-10', '2023-03-08', '2023-01-20', '2023-02-15', '2023-03-10', 
        '2023-01-25', '2023-02-17', '2023-03-15'
    ],
    'Quantity': [20, 10, 25, 20, 30, 18, 30, 25, 50, 18, 15, 20, 30, 15, 40, 23, 10, 10]
}


# Convert purchase details to DataFrame
purchase_df = pd.DataFrame(purchase_details)

# Fetch historical data for each stock
start = '2023-01-01'
end = datetime.now().strftime('%Y-%m-%d')
stock_data = {}
for stock in stocks:
    ticker = yf.Ticker(stock)
    stock_data[stock] = ticker.history(start=start, end=end)['Close']

# Convert to DataFrame
stock_prices_df = pd.DataFrame(stock_data)

# Get purchase prices
purchase_prices = []
for index, row in purchase_df.iterrows():
    stock = row['Stock']
    purchase_date = row['Date of Purchase']
    if purchase_date in stock_prices_df.index:
        purchase_price = stock_prices_df.loc[purchase_date, stock]
    else:
        purchase_price = None
    purchase_prices.append(purchase_price)

# Add purchase prices to DataFrame
purchase_df['Purchase Price'] = purchase_prices

# Get current prices, forward P/E, std deviation, and sector
current_prices = []
forward_pe = []
std_dev = []
sectors = []
for stock in stocks:
    ticker = yf.Ticker(stock)
    current_price = ticker.info.get('currentPrice')
    forward_pe_value = ticker.info.get('forwardPE')
    std_dev_value = ticker.history(period='1y')['Close'].std() if 'Close' in ticker.history(period='1y') else None
    sector = ticker.info.get('sector')
    
    current_prices.append(current_price)
    forward_pe.append(forward_pe_value)
    std_dev.append(std_dev_value)
    sectors.append(sector)

# Add information to DataFrame
purchase_df['Current Price'] = current_prices
purchase_df['Forward P/E'] = forward_pe
purchase_df['Standard Deviation'] = std_dev
purchase_df['Sector'] = sectors

# Calculate invested value and current value
purchase_df['Invested Value'] = purchase_df['Quantity'] * purchase_df['Purchase Price']
purchase_df['Current Value'] = purchase_df['Quantity'] * purchase_df['Current Price']

# Calculate profit/loss and yield
purchase_df['Profit/Loss'] = purchase_df['Current Value'] - purchase_df['Invested Value']
purchase_df['Yield (%)'] = (purchase_df['Profit/Loss'] / purchase_df['Invested Value']) * 100

# Calculate total values
total_invested = purchase_df['Invested Value'].sum()
total_current = purchase_df['Current Value'].sum()
total_profit_loss = purchase_df['Profit/Loss'].sum()

if total_invested != 0:
    total_yield = (total_profit_loss / total_invested) * 100
else:
    total_yield = 0

# Calculate percentage of each stock in the portfolio
purchase_df['Percent of Total'] = (purchase_df['Current Value'] / total_current) * 100

# Add total row
totals = pd.DataFrame({
    'Stock': ['TOTAL'],
    'Date of Purchase': ['-'],
    'Quantity': ['-'],
    'Purchase Price': ['-'],
    'Current Price': ['-'],
    'Forward P/E': ['-'],
    'Standard Deviation': ['-'],
    'Sector': ['-'],
    'Invested Value': [total_invested],
    'Current Value': [total_current],
    'Profit/Loss': [total_profit_loss],
    'Yield (%)': [total_yield],
    'Percent of Total': [100]
})

purchase_df = pd.concat([purchase_df, totals], ignore_index=True)

totals = pd.DataFrame({
    'Stock': ['TOTAL'],
    'Date of Purchase': ['-'],
    'Quantity': ['-'],
    'Purchase Price': ['-'],
    'Current Price': ['-'],
    'Forward P/E': ['-'],
    'Standard Deviation': ['-'],
    'Sector': ['-'],
    'Invested Value': [total_invested],
    'Current Value': [total_current],
    'Profit/Loss': [total_profit_loss],
    'Yield (%)': [total_yield],
    'Percent of Total': [100]
})

purchase_df = pd.concat([purchase_df, totals], ignore_index=True)


# ==== Segunda Página: Dados dos Índices (S&P 500 e NASDAQ 100) ====
# Dados históricos dos índices
start_date = '2023-01-01'
end_date= datetime.now().strftime('%Y-%m-%d')
sp500_ticker = '^GSPC'
nasdaq100_ticker = '^NDX'

sp500_data = yf.Ticker(sp500_ticker).history(start=start, end=end)['Close']


nasdaq100_data = yf.Ticker(nasdaq100_ticker).history(start=start, end=end)['Close']

# Remover fuso horário dos índices
sp500_data.index = sp500_data.index.tz_localize(None)
nasdaq100_data.index = nasdaq100_data.index.tz_localize(None)

returns_sp500 = sp500_data.pct_change() * 100  # Convert to percentage
returns_nasdaq100 = nasdaq100_data.pct_change()*100

index_df = pd.DataFrame({
     'S&P 500': sp500_data,
    'S&P 500 Daily Return (%)': returns_sp500,
    'NASDAQ 100': nasdaq100_data,
    'NASDAQ 100 Daily Return (%)': returns_nasdaq100, 
    'sp500_yield' : (returns_sp500.iloc[-1] / returns_sp500.iloc[0] - 1) * 100,
    'nasdaq100_yield' : (returns_nasdaq100.iloc[-1] / returns_nasdaq100.iloc[0] - 1) * 100
})

# ==== Terceira Página: Dados Macroeconômicos ====
# Inicializando a API do FRED
fred = Fred(api_key='2bb1d9c9270f876de57d65a8c7fb97a7')

# Dados econômicos
start_date = '2023-01-01'
end_date = datetime.now().strftime('%Y-%m-%d')

def filter_dates(df, start, end):
    """Filtra as séries temporais para incluir apenas datas no intervalo especificado."""
    return df[(df.index >= start) & (df.index <= end)]

cpi_data = fred.get_series('CPIAUCNS', start_date=start_date, end_date=end_date).pct_change() * 100
ppi_data = fred.get_series('PPIACO', start_date=start_date, end_date=end_date).pct_change() * 100
unemployment_rate = fred.get_series('UNRATE', start_date=start_date, end_date=end_date)
treasury_10y_yield = fred.get_series('GS10', start_date=start_date, end_date=end_date)
interest_rate = fred.get_series('FEDFUNDS', start_date=start_date, end_date=end_date)

# Remover fuso horário dos dados macroeconômicos
cpi_data.index = cpi_data.index.tz_localize(None)
ppi_data.index = ppi_data.index.tz_localize(None)
unemployment_rate.index = unemployment_rate.index.tz_localize(None)
treasury_10y_yield.index = treasury_10y_yield.index.tz_localize(None)
interest_rate.index = interest_rate.index.tz_localize(None)

cpi_data = filter_dates(cpi_data, start_date, end_date)
ppi_data = filter_dates(ppi_data, start_date, end_date)
unemployment_rate = filter_dates(unemployment_rate, start_date, end_date)
treasury_10y_yield = filter_dates(treasury_10y_yield, start_date, end_date)
interest_rate = filter_dates(interest_rate, start_date, end_date)

# Criar DataFrame com os dados econômicos
macro_df = pd.DataFrame({
    'Date': cpi_data.index,
    'CPI Change (%)': cpi_data.values,
    'PPI Change (%)': ppi_data.reindex(cpi_data.index).values,
    'Unemployment Rate (%)': unemployment_rate.reindex(cpi_data.index).values,
    'Treasury 10Y Yield (%)': treasury_10y_yield.reindex(cpi_data.index).values,
    'Interest Rate (%)': interest_rate.reindex(cpi_data.index).values
})

# ==== Salvar no Excel ====
with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
    purchase_df.to_excel(writer, sheet_name='Portfolio', index=False)
    index_df.to_excel(writer, sheet_name='benchmark', index=True)
    macro_df.to_excel(writer, sheet_name='Macroeconomic Data', index=False)

print(f"O arquivo foi salvo com sucesso em: {file_path}")


O arquivo foi salvo com sucesso em: C:\Users\Danielf\Desktop\projects data Daniel\Portfolio manager with python\overview portfolio.xlsx


In [3]:
# calculate the return in the index
file_path = r'C:\Users\Danielf\Desktop\projects data Daniel\Portfolio manager with python\overview portfolio.xlsx'
benchmark_df = pd.read_excel(file_path, sheet_name='benchmark')
start_date = '2023-01-01'
end_date = datetime.now().strftime('%Y-%m-%d')

# Fetch historical data for S&P 500 and NASDAQ 100
sp500_data = yf.Ticker('^GSPC').history(start=start_date, end=end_date)['Close']
nasdaq100_data = yf.Ticker('^NDX').history(start=start_date, end=end_date)['Close']

# Calculate total return for S&P 500 and NASDAQ 100 over the period
sp500_return = (sp500_data.iloc[-1] / sp500_data.iloc[0] - 1) * 100
nasdaq100_return = (nasdaq100_data.iloc[-1] / nasdaq100_data.iloc[0] - 1) * 100

benchmark_df.loc[0, 'Return S&P 500'] = 'Return S&P 500'
benchmark_df.loc[1, 'Return S&P 500'] = sp500_return
benchmark_df.loc[0, 'Return NASDAQ 100'] = 'Return NASDAQ 100'
benchmark_df.loc[1, 'Return NASDAQ 100'] = nasdaq100_return

benchmark_df.drop(benchmark_df.columns[[0, 1]], axis=1, inplace=True)


# Save the updated benchmark_df back to the Excel file
with pd.ExcelWriter(file_path, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    benchmark_df.to_excel(writer, sheet_name='benchmark', index=False)


In [5]:
from openpyxl.drawing.image import Image
#charts in the reports
# File path
file_path = r'C:\Users\Danielf\Desktop\projects data Daniel\Portfolio manager with python\overview portfolio.xlsx'

# ==== 1. Line Chart: Portfolio vs. Index Performance ====
# Recalculate returns for the portfolio
portfolio_initial_value = purchase_df['Invested Value'].sum()
portfolio_returns = purchase_df['Current Value'].sum() / portfolio_initial_value - 1

# Create a cumulative returns DataFrame
cumulative_returns_df = pd.DataFrame({
    'Date': stock_prices_df.index,
    'Portfolio': (stock_prices_df.mean(axis=1) / stock_prices_df.mean(axis=1).iloc[0] - 1) * 100,
    'S&P 500': (sp500_data / sp500_data.iloc[0] - 1) * 100,
    'NASDAQ 100': (nasdaq100_data / nasdaq100_data.iloc[0] - 1) * 100
})

# Plot the line chart for cumulative returns
plt.figure(figsize=(10, 6))
plt.plot(cumulative_returns_df['Date'], cumulative_returns_df['Portfolio'], label='Portfolio')
plt.plot(cumulative_returns_df['Date'], cumulative_returns_df['S&P 500'], label='S&P 500')
plt.plot(cumulative_returns_df['Date'], cumulative_returns_df['NASDAQ 100'], label='NASDAQ 100')

# Add labels and legend
plt.xlabel('Date')
plt.ylabel('Cumulative Return (%)')
plt.title('Portfolio vs. Index Performance')
plt.legend()
plt.grid(True)

# Adding percentage labels every 3 months 
for i in range(0, len(cumulative_returns_df), 90):
    plt.text(cumulative_returns_df['Date'][i], cumulative_returns_df['Portfolio'][i], 
             f"{cumulative_returns_df['Portfolio'][i]:.1f}%", fontsize=9, ha='right', color='blue')
    plt.text(cumulative_returns_df['Date'][i], cumulative_returns_df['S&P 500'][i], 
             f"{cumulative_returns_df['S&P 500'][i]:.1f}%", fontsize=9, ha='right', color='orange')
    plt.text(cumulative_returns_df['Date'][i], cumulative_returns_df['NASDAQ 100'][i], 
             f"{cumulative_returns_df['NASDAQ 100'][i]:.1f}%", fontsize=9, ha='right', color='green')

# Save the line chart as an image
plt.tight_layout()
plt.savefig(r'C:\Users\Danielf\Desktop\projects data Daniel\Portfolio manager with python\line_chart_portfolio_vs_index.png')
plt.close()

# ==== 2. Pie Chart: Sector Allocation ====
# Prepare the data for the pie chart
sector_allocation = purchase_df[purchase_df['Sector'] != '-'].groupby('Sector')['Current Value'].sum()

# Plot the pie chart
plt.figure(figsize=(8, 8))
plt.pie(sector_allocation, labels=sector_allocation.index, autopct='%1.1f%%', startangle=140)
plt.title('Portfolio Sector Allocation')

# Save the pie chart as an image
plt.savefig(r'C:\Users\Danielf\Desktop\projects data Daniel\Portfolio manager with python\pie_chart_sector_allocation.png')
plt.close()

# ==== 3. Line Charts: Macroeconomic Data ====
# Interest Rate Line Chart
plt.figure(figsize=(10, 6))
plt.plot(macro_df['Date'], macro_df['Interest Rate (%)'], label='Interest Rate (%)', color='red')
plt.xlabel('Date')
plt.ylabel('Interest Rate (%)')
plt.title('Interest Rate Over Time')
plt.grid(True)

# Adding percentage labels every 3 months
for i in range(0, len(macro_df), 90):  # Aproximadamente a cada 3 meses
    plt.text(macro_df['Date'][i], macro_df['Interest Rate (%)'][i], 
             f"{macro_df['Interest Rate (%)'][i]:.1f}%", fontsize=9, ha='right')

plt.tight_layout()
plt.savefig(r'C:\Users\Danielf\Desktop\projects data Daniel\Portfolio manager with python\line_chart_interest_rate.png')
plt.close()


# ==== Insert Charts into Excel ====
# Load existing workbook
wb = openpyxl.load_workbook(file_path)

# Create new sheets for the charts
wb.create_sheet('Portfolio vs Index')
wb.create_sheet('Sector Allocation')
wb.create_sheet('Macroeconomic Data chart')

# Insert images into Excel sheets
# Portfolio vs Index
ws1 = wb['Portfolio vs Index']
img1 = Image(r'C:\Users\Danielf\Desktop\projects data Daniel\Portfolio manager with python\line_chart_portfolio_vs_index.png')
ws1.add_image(img1, 'A1')

# Sector Allocation
ws2 = wb['Sector Allocation']
img2 = Image(r'C:\Users\Danielf\Desktop\projects data Daniel\Portfolio manager with python\pie_chart_sector_allocation.png')
ws2.add_image(img2, 'A1')

# Macroeconomic Data
ws3 = wb['Macroeconomic Data chart']
img3 = Image(r'C:\Users\Danielf\Desktop\projects data Daniel\Portfolio manager with python\line_chart_interest_rate.png')
ws3.add_image(img3, 'A1')

# Save the updated Excel file with charts
wb.save(file_path)

print("Charts added to the Excel file successfully!")


C:\Users\Danielf\AppData\Local\Temp\ipykernel_19388\4190475556.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  plt.text(cumulative_returns_df['Date'][i], cumulative_returns_df['Portfolio'][i],
C:\Users\Danielf\AppData\Local\Temp\ipykernel_19388\4190475556.py:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  f"{cumulative_returns_df['Portfolio'][i]:.1f}%", fontsize=9, ha='right', color='blue')
C:\Users\Danielf\AppData\Local\Temp\ipykernel_19388\4190475556.py:36: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consisten

Charts added to the Excel file successfully!


In [6]:
#code to check the API

print(f"Última data disponível para CPI: {cpi_data.dropna().index[-1]}")
print(f"Última data disponível para PPI: {ppi_data.dropna().index[-1]}")
print(f"Última data disponível para taxa de desemprego: {unemployment_rate.dropna().index[-1]}")
print(f"Última data disponível para rendimento do Tesouro 10Y: {treasury_10y_yield.dropna().index[-1]}")
print(f"Última data disponível para taxa de juros: {interest_rate.dropna().index[-1]}")

Última data disponível para CPI: 2024-08-01 00:00:00
Última data disponível para PPI: 2024-08-01 00:00:00
Última data disponível para taxa de desemprego: 2024-08-01 00:00:00
Última data disponível para rendimento do Tesouro 10Y: 2024-08-01 00:00:00
Última data disponível para taxa de juros: 2024-08-01 00:00:00


In [7]:
#CHECK FREQUENCY OF THE API REQUEST
cpi_info = fred.get_series_info('CPIAUCNS')
ppi_info = fred.get_series_info('PPIACO')
unemployment_info = fred.get_series_info('UNRATE')
treasury_info = fred.get_series_info('GS10')
interest_rate_info = fred.get_series_info('FEDFUNDS')

print(f"CPI Frequency: {cpi_info['frequency']}")
print(f"PPI Frequency: {ppi_info['frequency']}")
print(f"Unemployment Rate Frequency: {unemployment_info['frequency']}")
print(f"Treasury 10Y Yield Frequency: {treasury_info['frequency']}")
print(f"Interest Rate Frequency: {interest_rate_info['frequency']}")


CPI Frequency: Monthly
PPI Frequency: Monthly
Unemployment Rate Frequency: Monthly
Treasury 10Y Yield Frequency: Monthly
Interest Rate Frequency: Monthly
